In [1]:
import numpy as np
from sympy import *

In [2]:
x1, x2, x3 = symbols('x1 x2 x3')

# Define the function
f = 1/2*(10*x1**2+1*x2**2+10*x3**2)+5*log(1+exp(-x1-x2-x3))
# Define the gradient of f
df = Matrix([diff(f, x1), diff(f, x2), diff(f, x3)])
hf = df.jacobian([x1, x2, x3])
hf = lambdify((x1, x2, x3), hf)
df = lambdify((x1, x2, x3), df)

# Define the inequality constraints
A_ = np.array([[1, 0, 0], [0, 1, 1]])
b_ = Matrix(np.array([2, 0]))
h = A_ @ Matrix([x1, x2, x3]) - b_
dh = h.jacobian([x1, x2, x3])
dh = lambdify((x1, x2, x3), dh)
h = lambdify((x1, x2, x3), h)


# Define the equality constraints
A = np.array([[1 ,1, 0], [0, 1,0]])
b = np.array([0.2, 1.2])




In [3]:
def LHS(x, u, v, A=A, h=h, dh=dh, hx=hf):
    diag_u = np.diag(u)
    h = h(x[0],x[1],x[2])
    h = h.reshape(-1)
    diag_h = np.diag(h)
    dhx = dh(x[0],x[1],x[2])
    hx = hx(x[0],x[1],x[2])

    return np.block([[hx, dhx.T, A.T],[-diag_u @ dhx,-diag_h,np.zeros((h.shape[0],A.shape[0]))],[A,np.zeros((A.shape[0], h.shape[0])), np.zeros((A.shape[0],A.shape[0]))]])

def RHS(x, u, v, t, A=A, h=h, dh=dh, df=df):
    rdual = df(x[0],x[1],x[2]).reshape(-1) + dh(x[0],x[1],x[2]).T @ u + A.T @ v
    rcent = np.array(-np.diag(u) @ h(x[0],x[1],x[2]).reshape(-1)-1/t*np.ones(u.shape))
    rpime = A@x - b
    return np.block([-rdual, -rcent, -rpime])

In [4]:
beta = 0.8
alpha = 0.1
x0 = np.array([0, 0, 0])
u0 = np.array([1, 1])
v0 = np.array([1, 1])
ita0 = -h(x0[0],x0[1],x0[2]).reshape(-1)@u0
mu0 = 10
m = A.shape[0]
t0 = mu0*m/ita0
path = []
while True:
    path.append(x0)
    x = np.array([x0[0],x0[1],x0[2],u0[0],u0[1],v0[0],v0[1]])
    LHS_ = LHS(x0, u0, v0)
    RHS_ = RHS(x0, u0, v0, t0)
    delta = np.linalg.solve(LHS_, RHS_)
    dx = delta[:3]
    du = delta[3:5]
    dv = delta[5:7]
    if np.all(du>0):
        s_max = 1
    else:
        s_max = np.min([1, np.min(-u0[du<0]/du[du<0])])
    s = s_max
    # line search
    while True:
        if np.all(h(x0[0]+s*dx[0],x0[1]+s*dx[1],x0[2]+s*dx[2])<0):
            x1 = x0 + s*dx
            u1 = u0 + s*du
            v1 = v0 + s*dv
            new_res = RHS(x1, u1, v1, t0)
            if np.linalg.norm(new_res)<(1-alpha*s)*np.linalg.norm(RHS_):
                break
        else:
            s = beta*s
    print(s)
    # Newton update
    x0, u0, v0 = x1, u1, v1
    ita = -h(x0[0],x0[1],x0[2]).reshape(-1)@u0
    if ita<1e-6 and np.linalg.norm(RHS_[:3])**2+np.linalg.norm(RHS_[5:7])**2<1e-6:
        break
    t0 = mu0*m/ita

0.6896551724137924
0.8
1.0
1.0
1.0
1.0
1.0


In [5]:
path

[array([0, 0, 0]),
 array([-0.68965517,  0.82758621, -0.89655172]),
 array([-0.93793103,  1.12551724, -1.1257623 ]),
 array([-1.        ,  1.2       , -1.20011944]),
 array([-1.        ,  1.2       , -1.20000997]),
 array([-1.      ,  1.2     , -1.200001]),
 array([-1.       ,  1.2      , -1.2000001])]